In [ ]:
import pandas as pd
import numpy as np
import random
import json
from geopy.distance import vincenty


from matplotlib import pyplot as plt
%matplotlib inline


In [ ]:
usfd_files = {}
usfds1_dataframes = list()
time_intervals = ((1, 1996, 2000), (2, 2001, 2005), (3, 2006, 2010), (4, 2011, 2014))

for time_interval in time_intervals:
    filename = 'CX_DataScientistAssessment_20170713/{}.US_Fare_Dmd_{}_{}.xlsx'.format(*time_interval)
    usfd_file = pd.ExcelFile(filename)
    if usfd_file.sheet_names != ['Sheet1', 'Sheet2', 'Sheet3']:
        raise
    for dfsheet in [usfd_file.parse(_) for _ in ['Sheet2', 'Sheet3']]:
        if not dfsheet.empty:
            raise
    usfds1 = usfd_file.parse('Sheet1')
    usfds1['mincity'] = usfds1.apply(lambda _: min(_['City1'], _['City2']), axis=1)
    usfds1['maxcity'] = usfds1.apply(lambda _: max(_['City1'], _['City2']), axis=1)
    usfds1[['City1','City2']] = usfds1[['mincity', 'maxcity']]
    usfds1=usfds1.drop(['mincity', 'maxcity'], axis=1)
    usfds1['source'] = time_interval[0]
    usfds1_dataframes.append(usfds1)
    
try:
    for df in usfds1_dataframes:
        if not all(usfds1_dataframes[0].columns == df.columns):
            raise
except ValueError:
    raise

In [ ]:
usfd = pd.concat(usfds1_dataframes, axis=0, ignore_index=True)
usfd.to_pickle('usfd.pickle')
usfd.info()

In [ ]:
airline_type_file = pd.ExcelFile('CX_DataScientistAssessment_20170713/AirlineType.xlsx')
airline_types = airline_type_file.parse('AirlineType')
airline_types.index = airline_types['Airline']
del airline_types['Airline']
#airline_types[:5]

In [ ]:
markets_file = pd.ExcelFile('CX_DataScientistAssessment_20170713/2011_2014_Markets.xlsx')
markets = markets_file.parse('Sheet1')
markets['mincity'] = markets.apply(lambda _: min(_['city1'], _['city2']), axis=1)
markets['maxcity'] = markets.apply(lambda _: max(_['city1'], _['city2']), axis=1)
markets[['City1','City2']] = markets[['mincity', 'maxcity']]
markets=markets.drop(['mincity', 'maxcity', 'city1', 'city2'], axis=1)
#markets.info()


In [ ]:
#Showing that there are indeed missing values
grouped_markets = markets.groupby(['City1', 'City2']).mean()
grouped_markets.iloc[((grouped_markets.index.get_level_values('City1') == 'Albuquerque, NM') 
                     & (grouped_markets.index.get_level_values('City2') == 'Salt Lake City, UT'))
                    |((grouped_markets.index.get_level_values('City2') == 'Albuquerque, NM') 
                     & (grouped_markets.index.get_level_values('City1') == 'Salt Lake City, UT'))]

In [ ]:
average_airport_distance = markets.groupby(['City1', 'City2']).mean()
only_average_airport_distance = average_airport_distance[['distance']]
#only_average_airport_distance[:5]

In [ ]:
usfd_w_distance = pd.merge(left=usfd, right=only_average_airport_distance, how='left',
                           left_on=['City1', 'City2'], 
                           right_index=True)
#usfd_w_distance[:5]
#usfd_w_distance.info()

In [ ]:
usfd_w_distance_mssng = usfd_w_distance[pd.isnull(usfd_w_distance['distance'])]
lc = list(usfd_w_distance_mssng.groupby(['City1', 'City2']).size().index)
relevant_cities = set([_[0] for _ in lc ] + [_[1] for _ in lc])
relevant_cities.add('Dallas/Fort Worth, TX')
with open('relevant_cities.json','w') as f:
    f.write(json.dumps(list(relevant_cities)))
with open('citydict.json','r') as f:
    citydict = json.loads(f.read())

def distance_fix(row, cdict):
    if pd.isnull(row['distance'])\
        and row['City1'] in cdict \
        and row['City2'] in cdict:
            return vincenty(cdict[row['City1']], cdict[row['City2']]).miles
    return row['distance']
usfd_w_distance['distance'] = usfd_w_distance.apply(lambda row: distance_fix(row, citydict), axis=1)
#usfd_w_distance.info()

In [ ]:
usfd_w_distance_type = pd.merge(left=usfd_w_distance, right=airline_types, how='left',
                                left_on=['Airline'], right_index=True)
#usfd_w_distance_type[:5]

In [ ]:
airline_dict = {'TW': 'Legacy', #TWA
                'QQ': 'Regional', #Reno airlines
                'QX': 'Regional' #Horizon/Alaska
               }
def airline_type_fix(row):
    if pd.isnull(row['AirlineType']) and row['Airline'] in airline_dict:
            return airline_dict[row['Airline']]
    return row['AirlineType']
usfd_w_distance_type['AirlineType'] = usfd_w_distance_type.apply(airline_type_fix, axis=1)
usfd_w_distance_type_ok = usfd_w_distance_type[(pd.isnull(usfd_w_distance_type['AirlineType'])==False) & 
                                               (pd.isnull(usfd_w_distance_type['distance'])==False)].copy()
usfd_w_distance_type_ok.to_pickle('usfd_w_distance_type_ok.pickle')
#usfd_w_distance_type_ok.info()

In [ ]:
usfd_wdt = usfd_w_distance_type_ok.copy()
passed_time = lambda _: _['Year'] - 1996 + _['Quarter']/4 - .25
usfd_wdt['time'] = usfd_wdt.apply(passed_time, axis=1)
fare_spread = lambda _: _['MaxFare'] / _['MinFare']
usfd_wdt['fare_spread'] = usfd_wdt.apply(fare_spread, axis=1)
fare_per_km = lambda _: _['AvgFare'] / _['distance']
usfd_wdt['fare_per_km'] = usfd_wdt.apply(fare_per_km, axis=1)
against_market = lambda _: _['AvgFare'] / _['Avg_mkt_fare']
usfd_wdt['against_market'] = usfd_wdt.apply(against_market, axis=1)

In [ ]:
airline_type_presence = usfd_wdt.groupby(['Year', 'Quarter', 'City1', 'City2', 'AirlineType']).sum()
airline_type_presence_lcc = airline_type_presence.iloc[airline_type_presence.index.get_level_values('AirlineType') 
                                                       == 'LCC'].copy()
airline_type_presence_lcc = airline_type_presence_lcc[['MarketShare']]
airline_type_presence_lcc.columns = ['LCC_market_share']
airline_type_presence_lcc['has_lcc_presence'] = 1
has_strong_lcc = lambda _: 1 if _['LCC_market_share']>.3 else 0
airline_type_presence_lcc['has_strong_lcc'] = airline_type_presence_lcc.apply(has_strong_lcc, axis=1)
airline_type_presence_lcc.index = airline_type_presence_lcc.index.droplevel(['AirlineType'])
#airline_type_presence_lcc[:5]


In [ ]:
airline_competition = usfd_wdt.groupby(['Year', 'Quarter', 'City1', 'City2']).size()
airline_competition_frame = airline_competition.to_frame()
airline_competition_frame.columns = ['competitors']
#airline_competition_frame[:5]

In [ ]:
market_size = usfd_wdt.groupby(['Year', 'Quarter', 'City1', 'City2']).sum()['Passengers'].to_frame()
market_size.columns = ['market_size']
#market_size[:5]

In [ ]:
usfd_extended_0 = pd.merge(left=usfd_wdt, right=airline_type_presence_lcc, how='left',
                                left_on=['Year', 'Quarter', 'City1', 'City2'], right_index=True)
usfd_extended_0.fillna(0, inplace=True)

In [ ]:
usfd_extended_1 = pd.merge(left=usfd_extended_0, right=airline_competition_frame, how='left',
                                left_on=['Year', 'Quarter', 'City1', 'City2'], right_index=True)
#usfd_extended_1[:5]

In [ ]:
usfd_extended_2 = pd.merge(left=usfd_extended_1, right=market_size, how='left',
                                left_on=['Year', 'Quarter', 'City1', 'City2'], right_index=True)
#usfd_extended_2[:5]

In [ ]:
usfd_onehot = pd.concat([ usfd_extended_2,
                           pd.get_dummies(usfd_extended_2[['AirlineType']]),
                           pd.get_dummies(usfd_extended_2['Quarter'])],
                         axis=1).drop(['City1ID', 'City1', 'City2ID', 'City2', 
                                              'AirlineID', 'Airline', 'AirlineType'], axis=1)

usfd_city_onehot = pd.concat([ usfd_extended_2, 
                           pd.get_dummies(usfd_extended_2[['City1']]), 
                           pd.get_dummies(usfd_extended_2[['City2']]), 
                           pd.get_dummies(usfd_extended_2[['AirlineType']]),
                           pd.get_dummies(usfd_extended_2['Quarter'])],
                         axis=1).drop(['City1ID', 'City1', 'City2ID', 'City2', 
                                              'AirlineID', 'Airline', 'AirlineType'], axis=1)

usfd_onehot.to_pickle('usfd_onehot.pickle')
#usfd_onehot[:5]

In [ ]:
usfd_extended_additional = usfd_extended_2.copy()

In [ ]:
western_metro = {'Los Angeles, CA (Metropolitan Area)', 'San Francisco, CA (Metropolitan Area)', '',}
eastern_metro = {'Boston, MA (Metropolitan Area)', 'New York City, NY (Metropolitan Area)', 
                 'Miami, FL (Metropolitan Area)', 'Washington, DC (Metropolitan Area)'}
other_metro = {'Chicago, IL', 'Dallas/Fort Worth, TX', 'Atlanta, GA (Metropolitan Area)',  'Houston, TX'}
all_metro = western_metro | eastern_metro | other_metro

def is_transcon(row):
    return row['City1'] in western_metro and row['City2'] in eastern_metro \
        or row['City2'] in western_metro and row['City1'] in eastern_metro
    
def is_metro_metro(row):
    return row['City1'] in all_metro and row['City2'] in all_metro

def is_metro(row):
    return row['City1'] in all_metro or row['City2'] in all_metro

usfd_extended_additional['is_metro'] = usfd_extended_additional.apply(is_metro, axis=1)
usfd_extended_additional['is_metro_metro'] = usfd_extended_additional.apply(is_metro_metro, axis=1)
usfd_extended_additional['is_transcon'] = usfd_extended_additional.apply(is_transcon, axis=1)


In [ ]:
kerosene_file = pd.ExcelFile('CX_DataScientistAssessment_20170713/EMA_EPPK_PWG_NUS_DPGm.xls')
kerosene = kerosene_file.parse('Data 1')[2:]
kerosene.columns=['date','price']
kerosene['year'] = kerosene.date.dt.year
kerosene['quarter'] = kerosene.date.dt.quarter
kerosene.drop(['date'], axis=1)
kerosene['price'] = np.float32(kerosene['price'])
kerosene_by_quarter = kerosene.groupby(['year', 'quarter']).mean()
usfd_extended_additional_kerosene = pd.merge(how='left', left=usfd_extended_additional, right=kerosene_by_quarter, 
                                            left_on=['Year','Quarter'], right_index=True)

In [ ]:
usfd_additional_onehot = pd.concat([ usfd_extended_additional_kerosene, 
                           pd.get_dummies(usfd_extended_additional_kerosene[['AirlineType']]),
                           pd.get_dummies(usfd_extended_additional_kerosene['Quarter'])],
                         axis=1).drop(['City1ID', 'City1', 'City2ID', 'City2', 
                                              'AirlineID', 'Airline', 'AirlineType'], axis=1)

usfd_additional_city_onehot = pd.concat([ usfd_extended_additional_kerosene, 
                           pd.get_dummies(usfd_extended_additional_kerosene[['City1']]), 
                           pd.get_dummies(usfd_extended_additional_kerosene[['City2']]), 
                           pd.get_dummies(usfd_extended_additional_kerosene[['AirlineType']]),
                           pd.get_dummies(usfd_extended_additional_kerosene['Quarter'])],
                         axis=1).drop(['City1ID', 'City1', 'City2ID', 'City2', 
                                              'AirlineID', 'Airline', 'AirlineType'], axis=1)

In [ ]:
def write_df_pickle(df, filename):
    data_df = df.drop(['MaxShare'], axis=1)
    target_df = df[['MaxShare']]
    data_df.to_pickle('{}_data.pickle'.format(filename))
    target_df.to_pickle('{}_target.pickle'.format(filename))

In [ ]:
def write_by_time(df, name):
    train = df[df['Year'].isin(list(range(1996, 2011)))].drop(['Year'], axis=1)
    write_df_pickle(train, '{}_by_time_train'.format(name))
    test = df[df['Year'].isin(list(range(2011, 2016)))].drop(['Year'], axis=1)
    write_df_pickle(test, '{}_by_time_test'.format(name))
write_by_time(usfd_onehot, 'usfd_onehot')
write_by_time(usfd_city_onehot, 'usfd_city_onehot')
write_by_time(usfd_additional_onehot, 'usfd_additional_onehot')
write_by_time(usfd_additional_city_onehot, 'usfd_additional_city_onehot')



In [ ]:
def random_train_test_split(df, test_share):
    test_indices = random.sample(list(df.index), int(len(df)*test_share))
    train_indices = [_ for _ in df.index if _ not in test_indices]
    return df.loc[train_indices], df.loc[test_indices]

def write_random_20(df, name):
    random_train_set, random_test_set = random_train_test_split(df, .2)
    write_df_pickle(random_train_set, '{}_random_20_train'.format(name))
    write_df_pickle(random_test_set, '{}_random_20_test'.format(name))
write_random_20(usfd_onehot, 'usfd_onehot')
write_random_20(usfd_city_onehot, 'usfd_city_onehot')
write_random_20(usfd_additional_onehot, 'usfd_additional_onehot')
write_random_20(usfd_additional_city_onehot, 'usfd_additional_city_onehot')





